In [4]:
import os
from glob import glob
from pathlib import Path
import json

from pprint import pprint

import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Paths to data and results

In [12]:
# TODO: Change the following paths according to your own setup
DATA_ROOT = "/rvl-home/guqiao/ldata/adt_processed/"
RESULT_ROOT = "/rvl-home/guqiao/src/gaussian_splatting/output/adt_v3/"
GG_RESULT_ROOT = "/rvl-home/guqiao/src/gaussian-grouping/output/gg_adt_v2"

RESULT_ROOT = Path(RESULT_ROOT)
DATA_ROOT = Path(DATA_ROOT)
GG_RESULT_ROOT = Path(GG_RESULT_ROOT)

result_folders = glob(str(RESULT_ROOT / "*" / "*"))
print(f"Found {len(result_folders)} result folders")

gg_result_folders = glob(str(GG_RESULT_ROOT / "*"))
print(f"Found {len(gg_result_folders)} gg result folders")

# subset_shown = "1-valid"  # from the seen subset
subset_shown = "2-test"   # from the noval subset

Found 64 result folders
Found 16 gg result folders


### Load and aggregate the 2D segmentation results

In [7]:
dfs_all = []

'''
Load our own results
'''
for result_folder in result_folders:
    exp_name = result_folder.split("/")[-1]
    scene_name = result_folder.split("/")[-2]
    
    data_folder = DATA_ROOT / scene_name
    instance_json_path = data_folder / "instances.json"
    instance_json = json.load(open(instance_json_path))
    df_instance = pd.DataFrame([
        {
            "instance_id": v['instance_id'],
            "instance_name": v["instance_name"],
            "category": v["category"],
            "category_uid": v["category_uid"],
        } for k, v in instance_json.items()
    ])
    
    # Add the normal evaluation
    log_file_paths = glob(str(Path(result_folder) / "2dseg_eval" / "*.csv"))
    
    for log_file_path in log_file_paths:
        log_file_name = os.path.basename(log_file_path)[:-4]
        subset = log_file_name.split("_logs")[0]
        if "_logs_" in log_file_name:
            eval_mode = log_file_name.split("_logs_")[1]
        else:
            eval_mode = "default"
        if "gt_" in eval_mode:
            eval_mode = "gt"
        df_log = pd.read_csv(log_file_path)
        df_log["exp_name"] = exp_name
        df_log["scene_name"] = scene_name
        df_log['subset'] = subset
        df_log['eval_mode'] = eval_mode
        df_log = df_log.rename(columns={"seg_id": "instance_id"})
        df_log = df_log.merge(df_instance, on="instance_id", how="left")

        dfs_all.append(df_log)
        
    # Add the evaluation of SAM
    log_file_paths = glob(str(Path(result_folder) / "2dseg_eval_sam" / "*.csv"))
    for log_file_path in log_file_paths:
        log_file_name = os.path.basename(log_file_path)[:-4]
        subset = log_file_name.split("_logs")[0]
        if "_logs_" in log_file_name:
            eval_mode = log_file_name.split("_logs_")[1]
        else:
            eval_mode = "default"
        df_log = pd.read_csv(log_file_path)
        df_log["exp_name"] = "sam"
        df_log["scene_name"] = scene_name
        df_log['subset'] = subset
        df_log['eval_mode'] = eval_mode
        df_log = df_log.rename(columns={"seg_id": "instance_id"})
        df_log = df_log.merge(df_instance, on="instance_id", how="left")

        dfs_all.append(df_log)
        
    log_file_paths = glob(str(Path(result_folder) / "2dseg_eval_cross" / "*.csv"))
    for log_file_path in log_file_paths:
        log_file_name = os.path.basename(log_file_path)[:-4]
        subset = log_file_name.split("_logs")[0]
        if "_logs_" in log_file_name:
            eval_mode = log_file_name.split("_logs_")[1]
        else:
            eval_mode = "default"
        df_log = pd.read_csv(log_file_path)
        df_log["exp_name"] = exp_name
        df_log["scene_name"] = scene_name
        df_log['subset'] = subset
        df_log['eval_mode'] = "cross-" + eval_mode
        df_log = df_log.rename(columns={"seg_id": "instance_id"})
        df_log = df_log.merge(df_instance, on="instance_id", how="left")

        dfs_all.append(df_log)
        
'''
Load the Gaussian Grouping results
'''
for result_folder in gg_result_folders:
    exp_name = "gaussian_grouping"
    scene_name = result_folder.split("/")[-1]
    
    data_folder = DATA_ROOT / scene_name
    instance_json_path = data_folder / "instances.json"
    instance_json = json.load(open(instance_json_path))
    df_instance = pd.DataFrame([
        {
            "instance_id": v['instance_id'],
            "instance_name": v["instance_name"],
            "category": v["category"],
            "category_uid": v["category_uid"],
        } for k, v in instance_json.items()
    ])
    
    # Add the normal evaluation
    log_file_paths = glob(str(Path(result_folder) / "2dseg_eval" / "*.csv"))
    for log_file_path in log_file_paths:
        log_file_name = os.path.basename(log_file_path)[:-4]
        subset = log_file_name.split("_logs")[0]
        if "_logs_" in log_file_name:
            eval_mode = log_file_name.split("_logs_")[1]
        else:
            eval_mode = "default"
        if "gt_" in eval_mode:
            eval_mode = "gt"
        df_log = pd.read_csv(log_file_path)
        df_log["exp_name"] = exp_name
        df_log["scene_name"] = scene_name
        df_log['subset'] = subset
        df_log['eval_mode'] = eval_mode
        df_log = df_log.rename(columns={"seg_id": "instance_id"})
        df_log = df_log.merge(df_instance, on="instance_id", how="left")

        dfs_all.append(df_log)
        
    log_file_paths = glob(str(Path(result_folder) / "2dseg_eval_cross" / "*.csv"))
    for log_file_path in log_file_paths:
        log_file_name = os.path.basename(log_file_path)[:-4]
        subset = log_file_name.split("_logs")[0]
        if "_logs_" in log_file_name:
            eval_mode = log_file_name.split("_logs_")[1]
        else:
            eval_mode = "default"
        df_log = pd.read_csv(log_file_path)
        df_log["exp_name"] = exp_name
        df_log["scene_name"] = scene_name
        df_log['subset'] = subset
        df_log['eval_mode'] = "cross-" + eval_mode
        df_log = df_log.rename(columns={"seg_id": "instance_id"})
        df_log = df_log.merge(df_instance, on="instance_id", how="left")

        dfs_all.append(df_log)
        

df_all = pd.concat(dfs_all, axis=0)
df_all = df_all.reset_index(drop=True)

df = df_all

df['exp_name'] = df['exp_name'].replace({
    "sam": "1-sam",
    "gaussian_grouping": "2-gg",
    "unc_2d_unet_baseline_contr16_thresh0.5": "3-3dgs",
    "deform_def_contr16": "4-dynamic 3dgs",
    "unc_2d_unet_sigmoid_contr16_thresh0.5": "5-ours"
})

df['seg_type'] = df['seg_type'].replace({
    "static": "1-static",
    "dynamic": "2-dynamic",
    "all": "3-all"
})
df['subset'] = df['subset'].replace({
    "valid": "1-valid",
    "test": "2-test"
})

df['eval_mode'] = df['eval_mode'].replace({
    "gt": "1-gt",
    "cross-gt_0.6_5": "2-cross-gt_0.6_5"
})

df = df[df['eval_mode'] != "fixed_0.6"]

df = df[df['subset'].isin(['1-valid', '2-test'])]

df_all_seg = df.groupby(["subset", "exp_name", "eval_mode"]).agg({
    "iou": ["mean"],
})
df_all_seg.columns = df_all_seg.columns.droplevel(1)
df_all_seg['iou'] = df_all_seg['iou'] * 100
df_all_seg = df_all_seg.rename(columns={"iou": "mIoU"})
df_all_seg = df_all_seg.reset_index()
df_all_seg['seg_type'] = "3-all"


df = df.groupby(["subset", "exp_name", "eval_mode", "seg_type"]).agg({
    "iou": ["mean"],
})
# Drop the multi-level of columns
df.columns = df.columns.droplevel(1)

df['iou'] = df['iou'] * 100
df = df.rename(columns={"iou": "mIoU"})
df = df.reset_index()
df = df[::-1]

df = pd.concat([df_all_seg, df], axis=0)

df = df[df['subset'] == subset_shown]
df = df.sort_values(by="exp_name", ascending=False)
df = df.sort_values(by=["subset", 'eval_mode', "seg_type"], ascending=True)
# df = df.sort_values(by=, ascending=True)

df_2dseg = df

df = df.pivot(index=['exp_name'], columns=["subset", 'eval_mode', "seg_type"], values="mIoU")

# display float with 2 decimal places
with pd.option_context('display.float_format', '{:.2f}'.format):
    print(df)

print(df.to_latex(escape=False, float_format="%.2f"))

subset           2-test                                                 
eval_mode          1-gt                 2-cross-gt_0.6_5                
seg_type       1-static 2-dynamic 3-all         1-static 2-dynamic 3-all
exp_name                                                                
1-sam             54.51     32.77 50.69              NaN       NaN   NaN
2-gg              35.68     30.76 34.81            23.79     11.33 21.58
3-3dgs            55.67     39.61 52.86            51.29     18.67 45.49
4-dynamic 3dgs    54.23     38.62 51.49            51.10     18.02 45.22
5-ours            58.15     37.74 54.57            55.27     19.14 48.84
\begin{tabular}{lrrrrrr}
\toprule
subset & \multicolumn{6}{r}{2-test} \\
eval_mode & \multicolumn{3}{r}{1-gt} & \multicolumn{3}{r}{2-cross-gt_0.6_5} \\
seg_type & 1-static & 2-dynamic & 3-all & 1-static & 2-dynamic & 3-all \\
exp_name &  &  &  &  &  &  \\
\midrule
1-sam & 54.51 & 32.77 & 50.69 & NaN & NaN & NaN \\
2-gg & 35.68 & 30.76 & 34.8

### Load and aggregate 3D segmentation results

In [8]:
# Aggregate the results for 3D bounding box evaluation
dfs_all = []

# Load our own model
for result_folder in result_folders:
    exp_name = result_folder.split("/")[-1]
    scene_name = result_folder.split("/")[-2]
    
    data_folder = DATA_ROOT / scene_name
    instance_json_path = data_folder / "instances.json"
    instance_json = json.load(open(instance_json_path))
    df_instance = pd.DataFrame([
        {
            "instance_id": v['instance_id'],
            "instance_name": v["instance_name"],
            "category": v["category"],
            "category_uid": v["category_uid"],
        } for k, v in instance_json.items()
    ])
    
    log_file_paths = glob(str(Path(result_folder) / "3dbox_eval*" / "*.csv"))
    
    for log_file_path in log_file_paths:
        log_file_name = os.path.basename(log_file_path)[:-4]
        subset = log_file_name.split("_logs")[0]
        eval_mode = log_file_name.split("_logs_")[1]
        
        if "gt_" in eval_mode:
            eval_mode = "gt"
            
        eval_folder_name = log_file_path.split("/")[-2]
        if "cross" in eval_folder_name:
            eval_mode = "cross-" + eval_mode
            
        df_log = pd.read_csv(log_file_path)
        df_log["exp_name"] = exp_name
        df_log["scene_name"] = scene_name
        df_log['subset'] = subset
        df_log['eval_mode'] = eval_mode
        df_log = df_log.rename(columns={"seg_id": "instance_id"})
        df_log = df_log.merge(df_instance, on="instance_id", how="left")

        dfs_all.append(df_log)
        
# Load the Gaussian Grouping results
for result_folder in gg_result_folders:
    exp_name = "gaussian_grouping"
    scene_name = result_folder.split("/")[-1]
    
    data_folder = DATA_ROOT / scene_name
    instance_json_path = data_folder / "instances.json"
    instance_json = json.load(open(instance_json_path))
    df_instance = pd.DataFrame([
        {
            "instance_id": v['instance_id'],
            "instance_name": v["instance_name"],
            "category": v["category"],
            "category_uid": v["category_uid"],
        } for k, v in instance_json.items()
    ])
    
    log_file_paths = glob(str(Path(result_folder) / "3dbox_eval*" / "*.csv"))
    
    for log_file_path in log_file_paths:
        log_file_name = os.path.basename(log_file_path)[:-4]
        subset = log_file_name.split("_logs")[0]
        eval_mode = log_file_name.split("_logs_")[1]
        
        if "gt_" in eval_mode:
            eval_mode = "gt"
            
        eval_folder_name = log_file_path.split("/")[-2]
        if "cross" in eval_folder_name:
            eval_mode = "cross-" + eval_mode
            
        df_log = pd.read_csv(log_file_path)
        df_log["exp_name"] = exp_name
        df_log["scene_name"] = scene_name
        df_log['subset'] = subset
        df_log['eval_mode'] = eval_mode
        df_log = df_log.rename(columns={"seg_id": "instance_id"})
        df_log = df_log.merge(df_instance, on="instance_id", how="left")

        dfs_all.append(df_log)
    
        
df_all = pd.concat(dfs_all, axis=0)
df_all = df_all.reset_index(drop=True)

df = df_all

df['exp_name'] = df['exp_name'].replace({
    "sam": "1-sam",
    "unc_2d_unet_baseline_contr16_thresh0.5": "2-3dgs",
    "deform_def_contr16": "3-dynamic 3dgs",
    "gaussian_grouping": "4-gg",
    "unc_2d_unet_sigmoid_contr16_thresh0.5": "5-ours"
})

df = df[df['subset'].isin(['valid', 'test'])]

df = df.groupby(["subset", "exp_name", "eval_mode", "seg_type"]).agg({
    "iou": ["mean"],
})
# Drop the multi-level of columns
df.columns = df.columns.droplevel(1)

df['iou'] = df['iou'] * 100
df = df.rename(columns={"iou": "mIoU"})
df = df.reset_index()
df = df[::-1]
df = df.sort_values(by="exp_name", ascending=False)
df = df.sort_values(by=["subset", "seg_type"], ascending=False)

df = df.pivot(index=['eval_mode', 'exp_name'], columns=["subset", "seg_type"], values="mIoU")

# display float with 2 decimal places
with pd.option_context('display.float_format', '{:.2f}'.format):
    print(df)

subset                     test        
seg_type                 static dynamic
eval_mode exp_name                     
cross-gt  2-3dgs          21.10   17.92
          3-dynamic 3dgs  20.58   16.58
          4-gg             7.48    5.64
          5-ours          23.11   17.93
gt        2-3dgs          17.01   15.02
          3-dynamic 3dgs  15.96   16.91
          4-gg             9.43    9.69
          5-ours          17.75   16.79


### Load and aggregate photometric metrics (PSNR)

In [9]:
# Aggregate the PSNR evaluation logs
dfs_all = []
for result_folder in result_folders:
    exp_name = result_folder.split("/")[-1]
    scene_name = result_folder.split("/")[-2]
    
    log_file_paths = glob(str(Path(result_folder) / "eval_logs_*.csv"))
    
    for log_file_path in log_file_paths:
        log_file_name = os.path.basename(log_file_path)[:-4]
        subset = log_file_name.split("_logs_")[1]
        
        if subset not in ['valid', 'test']:
            continue
        
        df_log = pd.read_csv(log_file_path)
        df_log["exp_name"] = exp_name
        df_log["scene_name"] = scene_name
        df_log['subset'] = subset
        
        dfs_all.append(df_log)
        
for result_folder in gg_result_folders:
    exp_name = "gaussian_grouping"
    scene_name = result_folder.split("/")[-1]
    
    log_file_paths = glob(str(Path(result_folder) / "eval_logs_*.csv"))
    
    for log_file_path in log_file_paths:
        log_file_name = os.path.basename(log_file_path)[:-4]
        subset = log_file_name.split("_logs_")[1]
        
        if subset not in ['valid', 'test']:
            continue
        
        df_log = pd.read_csv(log_file_path)
        df_log["exp_name"] = exp_name
        df_log["scene_name"] = scene_name
        df_log['subset'] = subset
        
        dfs_all.append(df_log)
        
df_all = pd.concat(dfs_all, axis=0)
df_all = df_all.reset_index(drop=True)

df = df_all
df['exp_name'] = df['exp_name'].replace({
    "sam": "1-sam",
    "gaussian_grouping": "2-gg",
    "unc_2d_unet_baseline_contr16_thresh0.5": "3-3dgs",
    "deform_def_contr16": "4-dynamic 3dgs",
    "unc_2d_unet_sigmoid_contr16_thresh0.5": "5-ours"
})

df = df[~df['exp_name'].isin(["vanilla_vanilla"])]

df = df.groupby(["scene_name", "exp_name", "subset"]).agg({
    "static_psnr": ["mean"],
    "dyna_psnr": ["mean"],
    "psnr": ["mean"],
})
df.columns = df.columns.droplevel(1)

df = df.groupby(["exp_name", "subset"]).agg({
    "static_psnr": ["mean"],
    "dyna_psnr": ["mean"],
    "psnr": ["mean"],
})
df.columns = df.columns.droplevel(1)

df = df.reset_index()
df = df.melt(id_vars=['exp_name', 'subset'], value_vars=df.columns)
df = df.rename(columns={"variable": "seg_type", "value": "psnr"})
df['seg_type'] = df['seg_type'].replace({
    "static_psnr": "1-static",
    "dyna_psnr": "2-dynamic",
    "psnr": "3-all"
})
df['subset'] = df['subset'].replace({
    "valid": "1-valid",
    "test": "2-test"
})
df['eval_mode'] = "3-psnr"
df = df[df['subset'] == subset_shown]
df = df.sort_values(by=["subset", "seg_type"], ascending=True)

df_psnr = df

df = df.pivot(index=['exp_name'], columns=["subset", 'seg_type'], values="psnr")

# Find a row where "static_psnr", "dyna_psnr" or "psnr" is a string
# df = df[
#     (df['static_psnr'].apply(lambda x: isinstance(x, str))) |
#     (df['dyna_psnr'].apply(lambda x: isinstance(x, str))) |
#     (df['psnr'].apply(lambda x: isinstance(x, str))) 
# ]

with pd.option_context('display.float_format', '{:.2f}'.format):
    print(df)

print(df.to_latex(escape=False, float_format="%.2f"))

subset           2-test                
seg_type       1-static 2-dynamic 3-all
exp_name                               
2-gg              21.29     14.99 19.97
3-3dgs            21.37     15.32 20.16
4-dynamic 3dgs    21.16     15.39 19.93
5-ours            22.14     14.37 20.28
\begin{tabular}{llll}
\toprule
subset & \multicolumn{3}{r}{2-test} \\
seg_type & 1-static & 2-dynamic & 3-all \\
exp_name &  &  &  \\
\midrule
2-gg & 21.29 & 14.99 & 19.97 \\
3-3dgs & 21.37 & 15.32 & 20.16 \\
4-dynamic 3dgs & 21.16 & 15.39 & 19.93 \\
5-ours & 22.14 & 14.37 & 20.28 \\
\bottomrule
\end{tabular}



### Aggregate 2D segmentation and PSNR results and make Table 1

In [10]:
import math

df = pd.concat([df_2dseg, df_psnr], axis=0)
df = df.reset_index(drop=True)
# df['values'] = df['mIoU'] if 'mIoU' in df.columns else df['psnr']
# Perform the above line row by row
df['values'] = df.apply(lambda x: x['mIoU'] if not math.isnan(x['mIoU']) else x['psnr'], axis=1)

df = df.sort_values(by="exp_name", ascending=False)
df = df.sort_values(by=["subset", 'eval_mode', "seg_type"], ascending=True)

df = df.pivot(index=['exp_name'], columns=["subset", 'eval_mode', "seg_type"], values="values")

# display float with 2 decimal places
with pd.option_context('display.float_format', '{:.2f}'.format):
    print(df)

print(df.to_latex(escape=False, float_format="%.2f"))

subset           2-test                                                   \
eval_mode          1-gt                 2-cross-gt_0.6_5                   
seg_type       1-static 2-dynamic 3-all         1-static 2-dynamic 3-all   
exp_name                                                                   
1-sam             54.51     32.77 50.69              NaN       NaN   NaN   
2-gg              35.68     30.76 34.81            23.79     11.33 21.58   
3-3dgs            55.67     39.61 52.86            51.29     18.67 45.49   
4-dynamic 3dgs    54.23     38.62 51.49            51.10     18.02 45.22   
5-ours            58.15     37.74 54.57            55.27     19.14 48.84   

subset                                   
eval_mode        3-psnr                  
seg_type       1-static 2-dynamic 3-all  
exp_name                                 
1-sam               NaN       NaN   NaN  
2-gg              21.29     14.99 19.97  
3-3dgs            21.37     15.32 20.16  
4-dynamic 3dgs    21.